In [54]:
import pandas as pd
import numpy as np
from utils import *

In [55]:
# -- PRODUCT_FORECAST doeltabel --

# PRODUCT_FORECAST_ID
# PRODUCT_FORECAST_EXPECTED_VOLUME
# INVENTORY_LEVELS_DATA_INVENTORY_COUNT
# PRODUCT_PRODUCTION_COST
# PRODUCT_MARGIN
# YEAR_MONTH_YEAR (komt uit product_forecast)
# YEAR_MONTH_MONTH (komt uit product_forecast)
# PRODUCT_PRODUCT_NUMBER

In [56]:
go_sales = load_db('source/go_sales.sqlite')

In [57]:
product_forecast = load_csv('source/GO_SALES_PRODUCT_FORECASTData.csv')
inventory_levels = load_csv('source/GO_SALES_INVENTORY_LEVELSData.csv', np.arange(0,4))
product = load_table(go_sales, 'PRODUCT')

In [58]:
# Remove unnwanted columns
product = product[['PRODUCT_NUMBER', 'PRODUCTION_COST', 'MARGIN']]

# Transform product dtypes
product['PRODUCT_NUMBER'] = product['PRODUCT_NUMBER'].astype('int64')
product['PRODUCTION_COST'] = product['PRODUCTION_COST'].astype('float64')
product['MARGIN'] = product['MARGIN'].astype('float64')

# Add prefixes
product_forecast = product_forecast.add_prefix('PRODUCT_FORECAST_')
inventory_levels = inventory_levels.add_prefix('INVENTORY_LEVELS_')
product = product.add_prefix('PRODUCT_')


In [59]:
# Merge tables
# JOIN
# product_forecast_data.product_number = inventory_levels_data.product_number AND product_forecast_data.year = inventory_levels_data.inventory_year AND product_forecast_data.month = inventory_levels_data.inventory_month 

result = product_forecast.merge(inventory_levels, 
    left_on=['PRODUCT_FORECAST_PRODUCT_NUMBER', 'PRODUCT_FORECAST_YEAR', 'PRODUCT_FORECAST_MONTH'], 
    right_on=['INVENTORY_LEVELS_PRODUCT_NUMBER', 'INVENTORY_LEVELS_INVENTORY_YEAR', 'INVENTORY_LEVELS_INVENTORY_MONTH'])

result = result.merge(product, 
    left_on='PRODUCT_FORECAST_PRODUCT_NUMBER', 
    right_on='PRODUCT_PRODUCT_NUMBER')

result

,PRODUCT_FORECAST_PRODUCT_NUMBER,PRODUCT_FORECAST_YEAR,PRODUCT_FORECAST_MONTH,PRODUCT_FORECAST_EXPECTED_VOLUME,INVENTORY_LEVELS_INVENTORY_YEAR,INVENTORY_LEVELS_INVENTORY_MONTH,INVENTORY_LEVELS_PRODUCT_NUMBER,INVENTORY_LEVELS_INVENTORY_COUNT,PRODUCT_PRODUCT_NUMBER,PRODUCT_PRODUCTION_COST,PRODUCT_MARGIN
0,44,2022,12,383,2022,12,44,8922,44,231.11,0.33
1,45,2021,1,80,2021,1,45,1858,45,383.32,0.33
2,45,2021,2,51,2021,2,45,1930,45,383.32,0.33
3,45,2021,3,214,2021,3,45,1856,45,383.32,0.33
4,45,2021,4,300,2021,4,45,1676,45,383.32,0.33
...,...,...,...,...,...,...,...,...,...,...,...
3867,115,2022,8,282,2022,8,115,6654,115,5.00,0.50
3868,115,2022,9,920,2022,9,115,5634,115,5.00,0.50
3869,115,2022,10,1081,2022,10,115,4488,115,5.00,0.50
3870,115,2022,11,398,2022,11,115,4108,115,5.00,0.50


In [60]:
# Remove duplicate columns
result = result.drop(columns=['PRODUCT_FORECAST_PRODUCT_NUMBER', 'INVENTORY_LEVELS_PRODUCT_NUMBER', 'INVENTORY_LEVELS_INVENTORY_YEAR', 'INVENTORY_LEVELS_INVENTORY_MONTH'])

In [61]:
# Rename foreign key columns
result = result.rename(columns={
    'PRODUCT_FORECAST_YEAR': 'YEAR_MONTH_YEAR',
    'PRODUCT_FORECAST_MONTH': 'YEAR_MONTH_MONTH'
})

result

,YEAR_MONTH_YEAR,YEAR_MONTH_MONTH,PRODUCT_FORECAST_EXPECTED_VOLUME,INVENTORY_LEVELS_INVENTORY_COUNT,PRODUCT_PRODUCT_NUMBER,PRODUCT_PRODUCTION_COST,PRODUCT_MARGIN
0,2022,12,383,8922,44,231.11,0.33
1,2021,1,80,1858,45,383.32,0.33
2,2021,2,51,1930,45,383.32,0.33
3,2021,3,214,1856,45,383.32,0.33
4,2021,4,300,1676,45,383.32,0.33
...,...,...,...,...,...,...,...
3867,2022,8,282,6654,115,5.00,0.50
3868,2022,9,920,5634,115,5.00,0.50
3869,2022,10,1081,4488,115,5.00,0.50
3870,2022,11,398,4108,115,5.00,0.50
